<a href="https://colab.research.google.com/github/data602sps/datasetspractice/blob/main/07_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 7**

# **Weeks 8 & 9 - Pandas**
* In this homework assignment, you will explore and analyze a public dataset of your choosing. Since this assignment is “open-ended” in nature, you are free to expand upon the requirements below. However, you must meet the minimum requirments as indicated in each section.

* You must use Pandas as the **primary tool** to process your data.

* The preferred method for this analysis is in a .ipynb file. Feel free to use whichever platform of your choosing.  
 * https://www.youtube.com/watch?v=inN8seMm7UI (Getting started with Colab).

* Your data should need some "work", or be considered "dirty".  You must show your skills in data cleaning/wrangling.

### **Some data examples:**
•	https://www.data.gov/

•	https://opendata.cityofnewyork.us/

•	https://datasetsearch.research.google.com/

•	https://archive.ics.uci.edu/ml/index.php

### **Resources:**

•	https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

•	https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


### **Headings or comments**
**You are required to make use of comments, or headings for each section.  You must explain what your code is doing, and the results of running your code.**  Act as if you were giving this assignment to your manager - you must include clear and descriptive information for each section.

### **You may work as a group or indivdually on this assignment.**


# Introduction

In this section, please describe the dataset you are using.  Include a link to the source of this data.  You should also provide some explanation on why you choose this dataset.

The dataset I chose is *Emergency Response Incidents* from NYC OpenData: https://data.cityofnewyork.us/Public-Safety/Emergency-Response-Incidents/pasr-j7fb/about_data

I also uploaded the data to my personal GitHub repo: https://raw.githubusercontent.com/gillianmcgovern0/cuny-data-602/refs/heads/main/Emergency_Response_Incidents_20250422.csv

I chose this dataset because I not only thought it was interesting, but also it's a good example for cleaning data. This dataset has missing values and also a column that could be broken up into 2 separate columns. It also contains a date column.

Additionally, a lot of statistics can be performed with this data.

______________
# Data Exploration
Import your dataset into your .ipynb, create dataframes, and explore your data.  

Include:

* Summary statistics means, medians, quartiles,
* Missing value information
* Any other relevant information about the dataset.  



In [20]:
import pandas as pd

In [21]:
# Import the data and create the dataframe
df = pd.read_csv("https://raw.githubusercontent.com/gillianmcgovern0/cuny-data-602/refs/heads/main/Emergency_Response_Incidents_20250422.csv")

In [22]:
# Get summary of the data
print(df.head())
print(df.info())

                  Incident Type                       Location    Borough  \
0            Utility-Water Main               136-17 72 Avenue     Queens   
1  Structural-Sidewalk Collapse                   927 Broadway  Manhattan   
2                 Utility-Other                            NaN  Manhattan   
3          Administration-Other  Seagirt Blvd & Beach 9 Street     Queens   
4         Law Enforcement-Other                            NaN   Brooklyn   

            Creation Date Closed Date   Latitude  Longitude  
0  01/16/2017 01:13:38 PM         NaN  40.714004 -73.829989  
1  10/29/2016 12:13:31 PM         NaN  40.714422 -74.006076  
2  11/22/2016 08:53:17 AM         NaN        NaN        NaN  
3  11/14/2016 03:53:54 PM         NaN  40.714004 -73.829989  
4  10/29/2016 05:35:28 PM         NaN        NaN        NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11750 entries, 0 to 11749
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------  

There are 11,750 observations in this dataset, i.e. 11,750 rows. Each observation represents an indiviual incident report. The table has 7 variables - a mixture of categorical and numerical. Only 3 columns, Incident Type, Borough, and Creation Date, do not have any missing data.

##### Missing Values Information


Here's a closer look at the missing data:

In [23]:
# Check for missing data
df.isnull().sum()

,0
Incident Type,0
Location,877
Borough,0
Creation Date,0
Closed Date,3108
Latitude,1625
Longitude,1625


In [24]:
percent_missing = df.isnull().sum() * 100 / len(df)
print(percent_missing)

Incident Type     0.000000
Location          7.463830
Borough           0.000000
Creation Date     0.000000
Closed Date      26.451064
Latitude         13.829787
Longitude        13.829787
dtype: float64


For Location, 7% is pretty small and wouldn't be a value to interpolate, so we can get rid of those missing values and still have a good portion of data.

For Closed Date, 26% missing data is pretty high, but this variable is not related to our analysis. We care more about the details of the incident and when they occured. That's probably the reason why there are so many missing values for this variable. So we can just remove this column as it is unrelated.

For longitude and latitude, 14% of the data is missing which is small, but substantial. It doesn't make sense to guess an exact location, so I would normally just drop these values (after finding out why the values are missing). For the pupose of this excercise and practicing imputing, I will impute these values.

##### Summary statistics



In [25]:
# Using describe() to calculate summary statistics
stats = df.describe()
print(stats)

# Calculating mean, median, and mode
mean_values = df['Latitude'].mean()
median_values = df['Latitude'].median()
mode_values = df['Latitude'].mode().iloc[0]

print("Latitude mean, median and mode:")
print(mean_values)
print(median_values)
print(mode_values)

long_mean_values = df['Longitude'].mean()
long_median_values = df['Longitude'].median()
long_mode_values = df['Longitude'].mode().iloc[0]

print("Longitude mean, median and mode:")
print(long_mean_values)
print(long_median_values)
print(long_mode_values)

           Latitude     Longitude
count  10125.000000  10125.000000
mean      40.729092    -73.927295
std        0.076811      1.473164
min       40.499285    -74.252067
25%       40.681834    -73.989753
50%       40.729650    -73.953722
75%       40.771359    -73.896183
max       40.912828     74.071800
Latitude mean, median and mode:
40.729092234068794
40.72964971051421
40.71442154062271
Longitude mean, median and mode:
-73.92729534768617
-73.95372198613069
-74.00607638041981


# Data Wrangling
Create a subset of your original data and perform the following.  

1. Modify multiple column names.

2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.

3. Fix missing and invalid values in data.

4. Create new columns based on existing columns or calculations.

5. Drop column(s) from your dataset.

6. Drop a row(s) from your dataset.

7. Sort your data based on multiple variables.

8. Filter your data based on some condition.

9. Convert all the string values to upper or lower cases in one column.

10. Check whether numeric values are present in a given column of your dataframe.

11. Group your dataset by one column, and get the mean, min, and max values by group.
  * Groupby()
  * agg() or .apply()

12. Group your dataset by two columns and then sort the aggregated results within the groups.

**You are free (and should) to add on to these questions.  Please clearly indicate in your assignment your answers to these questions.**

##### Modify multiple column names





In [26]:
# Create copy of the dataframe to make changes
df2 = df.copy()


# Renaming columns - use underscores and make lowercase
df2.rename(columns={'Incident Type': 'incident_type', 'Location': 'location', 'Borough': 'borough',
                   'Creation Date': 'creation_date', 'Closed Date': 'closed_date', 'Latitude': 'latitude',
                   'Longitude': 'longitude'}, inplace=True)

##### Look at the structure of your data

Let's first convert the date columns to datetime data type since those aren't strings. Let's then check for special characters for the columns that are strings. From our initial insight into the data, we know that the location columns contain dashes so we won't check for those (we'll deal with that later).

In [27]:
# Convert date cols to datetime type
df2['creation_date'] = pd.to_datetime(df2['creation_date'])
df2['closed_date'] = pd.to_datetime(df2['closed_date'])


# Check for any special characters
print(df2[df2.incident_type.str.contains(r'[@#&$%+/*]')].incident_type)

df_test = df2.dropna(subset=['location'])
print(df_test[df_test.location.str.contains(r'[@#$%+/*]')].location)
print(df2[df2.borough.str.contains(r'[@#&$%+-/*]')].borough)

<ipython-input-27-834fe73e408d>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['creation_date'] = pd.to_datetime(df2['creation_date'])


7476    Structural-Settlement/Sinkhole Condition
8067      Utility-Internal Water/Steam Condition
8709                   Medical-Spray/Larvaciding
8714                   Medical-Spray/Larvaciding
8730                   Medical-Spray/Larvaciding
8982                   Medical-Spray/Larvaciding
9182     Utility-Damaged Sidewalk/ Exposed Wires
9269                   Medical-Spray/Larvaciding
9827                   Medical-Spray/Larvaciding
Name: incident_type, dtype: object
133         St.Nicholas Avenue c/s West 162 Street
410            4683 Park Avenue c/s E 188th street
1781                 135th Street c/s 109th Avenue
1869           4683 Park Avenue c/s E 188th street
2494           4683 Park Avenue c/s E 188th street
2709        St.Nicholas Avenue c/s West 162 Street
3567        St.Nicholas Avenue c/s West 162 Street
3749                Hanford Street c/s Maryland RD
3752              531 Clove Road c/s Forest Avenue
3815           4683 Park Avenue c/s E 188th street
4188          

Looks like all the special characters are expected and don't need to be removed.

##### Fix missing and invalid values in data



We know from our initial insight that `closed_date` has a lot of missing data, but since that column is unnecessary for this analysis, let's just remove that column.

Next we have `location`. Location is so specific, we won't be able to do much useful aggregation, so we should replace the missing values with "Unknown". That way we can keep the data.

For `longitude` and `latitude`, let's impute missing data with the mean just for the sake of practicing imputing.

In [28]:
# Drop the Closed Date since it's an unecessary column
df3 = df2.drop(columns=['closed_date'])

# Replace missing location data with 'Unknown'
df3[['location']] = df3[['location']].fillna(value='Unknown')

# Replace missing lat and long values with the mean
df3['latitude'] = df3['latitude'].fillna(df3['latitude'].mean())
df3['longitude'] = df3['longitude'].fillna(df3['longitude'].mean())

# Updated summary
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11750 entries, 0 to 11749
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   incident_type  11750 non-null  object        
 1   location       11750 non-null  object        
 2   borough        11750 non-null  object        
 3   creation_date  11750 non-null  datetime64[ns]
 4   latitude       11750 non-null  float64       
 5   longitude      11750 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 550.9+ KB
None


We are now left with 11,750 observations, without any missing data.

##### Create new columns based on existing columns or calculations

`incident_type` values have 2 parts - the general incident category and more detailed incident name. These 2 parts are separated by a - so let's split this data into 2 columns:

In [29]:
# Split incident_type by first occurence of "-"
df3[['incident_category', 'incident']] = df3['incident_type'].str.split('-', n=1, expand=True)
df3.head()

,incident_type,location,borough,creation_date,latitude,longitude,incident_category,incident
0,Utility-Water Main,136-17 72 Avenue,Queens,2017-01-16 13:13:38,40.714004,-73.829989,Utility,Water Main
1,Structural-Sidewalk Collapse,927 Broadway,Manhattan,2016-10-29 12:13:31,40.714422,-74.006076,Structural,Sidewalk Collapse
2,Utility-Other,Unknown,Manhattan,2016-11-22 08:53:17,40.729092,-73.927295,Utility,Other
3,Administration-Other,Seagirt Blvd & Beach 9 Street,Queens,2016-11-14 15:53:54,40.714004,-73.829989,Administration,Other
4,Law Enforcement-Other,Unknown,Brooklyn,2016-10-29 17:35:28,40.729092,-73.927295,Law Enforcement,Other


We now have 2 separate columns to represent the incident. In the next section we'll remove the original column since it's no longer needed.

##### Drop column(s) from your dataset

In [30]:
# Drop original incident_type
df3 = df3.drop(columns=['incident_type'])
df3.head()

,location,borough,creation_date,latitude,longitude,incident_category,incident
0,136-17 72 Avenue,Queens,2017-01-16 13:13:38,40.714004,-73.829989,Utility,Water Main
1,927 Broadway,Manhattan,2016-10-29 12:13:31,40.714422,-74.006076,Structural,Sidewalk Collapse
2,Unknown,Manhattan,2016-11-22 08:53:17,40.729092,-73.927295,Utility,Other
3,Seagirt Blvd & Beach 9 Street,Queens,2016-11-14 15:53:54,40.714004,-73.829989,Administration,Other
4,Unknown,Brooklyn,2016-10-29 17:35:28,40.729092,-73.927295,Law Enforcement,Other


We now have a workable dataframe.

##### Drop a row(s) from your dataset



Let's drop the first 5 rows:

In [31]:
# Drop first 5 rows via iloc
df3 = df3.iloc[5:]

##### Sort your data based on multiple variables



In [32]:
# Sort by creation_date and borough
df3.sort_values(by=['creation_date', 'borough'])

,location,borough,creation_date,latitude,longitude,incident_category,incident
7142,2111 Jerome Ave,Bronx,2011-05-04 06:03:58,40.854962,-73.906321,Fire,4th Alarm
7138,750 Baychester Ave.,Bronx,2011-05-04 06:09:36,40.874280,-73.833921,Utility,Transformer Fire
7139,Sutton Network,Manhattan,2011-05-04 07:46:18,40.729092,-73.927295,Utility,Electric Feeder Cable
7137,West 155 St & Amsterdam Ave,Manhattan,2011-05-04 09:29:07,40.831568,-73.942981,Structural,Construction Accident
11674,1711 Popham Ave,Bronx,2011-05-04 14:44:43,40.850122,-73.920078,Fire,2nd Alarm
...,...,...,...,...,...,...,...
3163,Unknown,Brooklyn,2022-07-19 02:49:32,40.729092,-73.927295,Utility,Power Outage
7495,Unknown,Brooklyn,2022-07-19 02:49:32,40.729092,-73.927295,Utility,Power Outage
2695,77 Gary Court,Staten Island,2022-08-12 21:12:35,40.608180,-74.160118,Fire,Other
2715,77 Gary Court,Staten Island,2022-08-12 21:12:35,40.608180,-74.160118,Fire,Other


##### Filter your data based on some condition



Filter for Queens incidents:

In [33]:
# Filter for only incidents which occured in Queens
df3[df3["borough"] == "Queens"]

,location,borough,creation_date,latitude,longitude,incident_category,incident
5,2-17 54 Avenue,Queens,2016-12-02 16:40:13,40.714004,-73.829989,Utility,Water Main
30,188 Street & 75 Avenue,Queens,2017-05-10 13:36:12,40.714004,-73.829989,Administration,Other
32,Unknown,Queens,2017-01-19 04:08:25,40.744196,-73.771544,Transportation,Other
33,155-26 115 Avenue,Queens,2016-12-17 09:04:03,40.729092,-73.927295,Fire,2nd Alarm
34,188 Street & 75 Avenue,Queens,2017-05-10 13:36:12,40.714004,-73.829989,Administration,Other
...,...,...,...,...,...,...,...
11717,173-12 93rd Avenue,Queens,2021-03-10 06:17:00,40.706180,-73.785535,Fire,2nd Alarm
11727,Unknown,Queens,2012-05-25 08:33:37,40.729092,-73.927295,Utility,Power Outage
11729,Unknown,Queens,2013-02-10 08:42:37,40.729092,-73.927295,Utility,Power Outage
11734,Unknown,Queens,2014-12-08 05:00:54,40.729092,-73.927295,Utility,Other


We can see this dataset has 1,962 observations.

##### Convert all the string values to upper or lower cases in one column



Let's convert `incident`, `incident_category` and `borough` to lowercase:

In [34]:
df3['incident'] = df3['incident'].str.lower()
df3['incident_category'] = df3['incident_category'].str.lower()
df3['borough'] = df3['borough'].str.lower()
df3.head()

,location,borough,creation_date,latitude,longitude,incident_category,incident
5,2-17 54 Avenue,queens,2016-12-02 16:40:13,40.714004,-73.829989,utility,water main
6,238 East 24 Street,manhattan,2016-11-25 04:06:09,40.714422,-74.006076,fire,2nd alarm
7,7th Avenue & West 27 Street,manhattan,2016-12-03 04:17:30,40.714422,-74.006076,utility,water main
8,130 East 57 Street,manhattan,2016-11-26 05:45:43,40.729092,-73.927295,fire,10-76 (commercial high rise fire)
9,Unknown,brooklyn,2016-11-18 13:12:51,40.729092,-73.927295,structural,crane


##### Check whether numeric values are present in a given column of your dataframe





`incident_category` shouldn't contain any numbers, so let's double check:

In [35]:
# Filter for any rows having numeric values
df3[df3.incident_category.str.isnumeric()]

,location,borough,creation_date,latitude,longitude,incident_category,incident


This proves that `incident_category` is numerical.

##### Group your dataset by one column, and get the mean, min, and max values by group

In [36]:
# Group by incident category and get the average latitude
df3.groupby(['incident_category'])[['latitude']].mean()

,latitude
incident_category,
administration,40.736142
aviation,40.705409
fire,40.732618
hazmat,40.730128
law enforcement,40.727590
lawenforcement,40.723608
marine,40.685194
medical,40.711474
other,40.709880


In [37]:
# Group by incident category and get the min latitude
df3.groupby(['incident_category'])[['latitude']].min()

,latitude
incident_category,
administration,40.573170
aviation,40.554941
fire,40.510709
hazmat,40.525864
law enforcement,40.517437
lawenforcement,40.517089
marine,40.561138
medical,40.572065
other,40.573623


In [38]:
# Group by incident category and get the max latitude
df3.groupby(['incident_category'])[['latitude']].max()

,latitude
incident_category,
administration,40.855084
aviation,40.795790
fire,40.912828
hazmat,40.905009
law enforcement,40.908179
lawenforcement,40.886851
marine,40.836151
medical,40.885552
other,40.881059


##### Group your dataset by two columns and then sort the aggregated results within the groups



In [39]:
# Group by borough and incident_category and aggregate longitude
df3.groupby(['borough','incident_category']).agg({'longitude':max})

<ipython-input-39-c335baccc2f9>:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df3.groupby(['borough','incident_category']).agg({'longitude':max})


longitude
borough                            incident_category           
astoria                            administration    -73.927295
                                   fire              -73.925182
bergen                             transportation    -73.927295
brinx                              fire              -73.916752
bronx                              administration    -73.886369
...                                                         ...
staten island                      transportation    -73.927295
                                   utility            74.071800
                                   weather           -73.927295
staten island (midland beach area) utility           -74.077802
the bronx                          fire              -73.853806

[147 rows x 1 columns]

We now have the max `longitude` by `borough` and `incident_category`.

##### Group your dataset by columns and get count

In [40]:
# Group by borough and get incident count in descending order
df3.groupby(['borough'])['incident_category'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)

,borough,Count
25,manhattan,3656
5,brooklyn,3444
41,queens,2054
3,bronx,1619
50,staten island,580
34,new york,275
15,jamaica,13
7,citywide,12
11,hoboken,9
9,far rockaway,7


We can see that Manhattan has the highest frequency of incident reports followed by Brooklyn and Queens. One thing I would try to do in the future is clean up these values and fix typos to get a more accurate count.

In [41]:
# Group by incident_category and get counts
df3.groupby(['incident_category'])['incident'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)

,incident_category,Count
2,fire,4233
13,utility,2440
11,structural,1886
4,law enforcement,854
3,hazmat,645
12,transportation,493
5,lawenforcement,334
10,rescue,280
0,administration,216
7,medical,170


We can see that fire, utility and structural are the top incident categories.

# Conclusions  

After exploring your dataset, provide a short summary of what you noticed from this dataset.  What would you explore further with more time?

This analysis showed:
* The mean, min, max latitude and longitude
* Top borough in NYC with incidents is Manhattan
* Top incident in NYC a fire

Opportunities for further analysis:
* Try imputing missing data via other methods like KNN possibly
* Map out where the incidents are by longitude and latitude data
* Look into time of year and frequency of incidents
* Work on fixing up typos in categorical variables